# Project: Human Resources Dataset Analysis

## Week 2: (1) Exploratory Data Analysis (EDA) And Determine Data Analysis Questions

In [1]:
# import Libraries
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc

pd.options.display.max_rows = None

pd.options.display.max_columns = None
sns.set()


In [2]:
import pyodbc
import pandas as pd

def read_sql_query(query):
    
    server = 'DESKTOP-0CQ5N9B' 
    database = 'HR_system' 
    
    # SQL Authentication
    connection_string = (
        f"Driver={{ODBC Driver 17 for SQL Server}};" 
        f"Server={server};" 
        f"Database={database};"
        f"Trusted_Connection=yes;" 
    )

    # Creating connection
    try:
        connection = pyodbc.connect(connection_string) 
        print("Connection successful!")

        # Use cursor to execute query
        cursor = connection.cursor()
        cursor.execute(query)
        
        # Get results
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description] 
        
        # Creating DataFrame
        df = pd.DataFrame.from_records(rows, columns=columns)

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None  
    
    finally:
        # close connection
        if 'connection' in locals():  
            connection.close()


# Employees

In [137]:
pth= "../00-Dataset_Data_Model/"
# employees = pd.read_csv(f"{pth}01-Employees.csv")
# educationlevels = pd.read_csv(f"{pth}03-EducationLevels.csv")


## Load Dataset of Employees with EducationLevels as categories
query = """ 
SELECT 
    E.EmployeeID,
    E.FirstName,
    E.LastName,
    E.Gender,
    E.Age,
    E.BusinessTravel,
    E.Department,
    E.DistanceFromHome,
    E.State,
    E.Ethnicity,
    E.EducationField,
    E.JobRole,
    E.MaritalStatus,
    E.Salary,
    E.StockOptionLevel,
    E.OverTime,
    E.HireDate,
    E.Attrition,
    E.YearsAtCompany,
    E.YearsInMostRecentRole,
    E.YearsSinceLastPromotion,
    E.YearsWithCurrManager,
    EL.EducationLevel AS EducationLevel
FROM 
    Employees E
LEFT JOIN 
    EducationLevels EL ON E.Education = EL.EducationLevelID;
"""

employees = read_sql_query(query)

employees.head()

Connection successful!


,EmployeeID,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome,State,Ethnicity,EducationField,JobRole,MaritalStatus,Salary,StockOptionLevel,OverTime,HireDate,Attrition,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager,EducationLevel
0,001A-8F88,Christy,Jumel,Male,22,Some Travel,Technology,40,CA,White,Information Systems,Software Engineer,Married,27763.00,0,No,2021-09-05,No,1,0,1,0,Masters
1,005C-E0FB,Fin,O'Halleghane,Non-Binary,24,Frequent Traveller,Sales,17,CA,White,Marketing,Sales Executive,Married,56155.00,1,No,2017-08-26,No,5,2,2,0,Masters
2,00A3-2445,Wyatt,Ziehm,Male,30,Some Travel,Technology,6,CA,Black or African American,Computer Science,Machine Learning Engineer,Married,126238.00,0,No,2012-03-08,No,10,3,6,6,High School
3,00B0-F199,Trueman,Jirasek,Male,23,Some Travel,Sales,35,CA,White,Marketing,Sales Executive,Married,97824.00,1,Yes,2020-03-16,Yes,1,0,1,0,No Formal Qualifications
4,00D4-DD53,Joyce,Goor,Female,30,Frequent Traveller,Technology,44,CA,Black or African American,Computer Science,Software Engineer,Single,68508.00,0,Yes,2012-01-28,Yes,5,4,4,4,No Formal Qualifications


In [147]:
employees["HireDate"]=pd.to_datetime(employees["HireDate"], format='%Y-%m-%d', errors='coerce')

In [148]:
# Display employees data information
employees.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   EmployeeID               1470 non-null   object        
 1   FirstName                1470 non-null   object        
 2   LastName                 1470 non-null   object        
 3   Gender                   1470 non-null   object        
 4   Age                      1470 non-null   int64         
 5   BusinessTravel           1470 non-null   object        
 6   Department               1470 non-null   object        
 7   DistanceFromHome         1470 non-null   int64         
 8   State                    1470 non-null   object        
 9   Ethnicity                1470 non-null   object        
 10  EducationField           1470 non-null   object        
 11  JobRole                  1470 non-null   object        
 12  MaritalStatus            1470 non-


## Summary statistics

In [149]:
numeric_employees = [
    "Age",
    "DistanceFromHome",
    "Salary",
    "StockOptionLevel",
    "OverTime",
    "YearsAtCompany",
    "YearsInMostRecentRole",
    "YearsSinceLastPromotion",
    "YearsWithCurrManager"
]

categorical_employees =[
    "Gender",
    "BusinessTravel",
    "Department",
    "State",
    "Ethnicity",
    "EducationField",
    "JobRole",
    "MaritalStatus",
    "Attrition",
    "EducationLevel"
]


In [150]:
# Summary statistics of numeric
employees[numeric_employees].describe()


,Age,DistanceFromHome,StockOptionLevel,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,28.989796,22.502721,0.793878,4.562585,2.293197,3.440816,2.239456
std,7.993055,12.811124,0.852077,3.288048,2.539093,2.945194,2.505774
min,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,12.000000,0.000000,2.000000,0.000000,1.000000,0.000000
50%,26.000000,22.000000,1.000000,4.000000,1.000000,3.000000,1.000000
75%,34.000000,33.000000,1.000000,7.000000,4.000000,6.000000,4.000000
max,51.000000,45.000000,3.000000,10.000000,10.000000,10.000000,10.000000


In [151]:
# Summary statistics of categorical
employees[categorical_employees].describe().T


,count,unique,top,freq
Gender,1470,4,Female,675
BusinessTravel,1470,3,Some Travel,1043
Department,1470,3,Technology,961
State,1470,3,CA,875
Ethnicity,1470,7,White,860
EducationField,1470,9,Computer Science,440
JobRole,1470,13,Sales Executive,327
MaritalStatus,1470,3,Married,624
Attrition,1470,2,No,1233
EducationLevel,1470,5,Bachelors,572


In [171]:
# Summary statistics of categorical
for cat_e in categorical_employees:
    print(f"Summary statistics of: {cat_e}")
    print((employees[cat_e].value_counts(normalize=True)*100).to_string(index=True, header=True,float_format='%.2f%%'))
    print("\n")  


Summary statistics of: Gender
Gender
Female              45.92%
Male                44.29%
Non-Binary           8.44%
Prefer Not To Say    1.36%


Summary statistics of: BusinessTravel
BusinessTravel
Some Travel          70.95%
Frequent Traveller   18.84%
No Travel            10.20%


Summary statistics of: Department
Department
Technology        65.37%
Sales             30.34%
Human Resources    4.29%


Summary statistics of: State
State
CA   59.52%
NY   28.50%
IL   11.97%


Summary statistics of: Ethnicity
Ethnicity
White                              58.50%
Black or African American          14.08%
Mixed or multiple ethnic groups    13.47%
Asian or Asian American             7.69%
American Indian or Alaska Native    3.40%
Native Hawaiian                     1.77%
Other                               1.09%


Summary statistics of: EducationField
EducationField
Computer Science      29.93%
Information Systems   24.69%
Marketing             11.29%
Marketing             10.82%
Economics  

__________________________________

## Summary 

____________________________

____________________________

__________________________

__________________________

### PerformanceRating:

In [89]:
# Reading Dataset of PerformanceRating.
#performancerating = pd.read_csv(f"{pth}02-PerformanceRating.csv")
#satisfactionlevels = pd.read_csv(f"{pth}04-SatisfactionLevels.csv")
#ratinglevels = pd.read_csv(f"{pth}05-RatingLevels.csv")


## Load Dataset of PerformanceRating with SatisfactionLevels and RatingLevels as categories
query = """ 
SELECT 
    PR.PerformanceID,
    PR.EmployeeID,
    PR.ReviewDate,
    PR.TrainingOpportunitiesWithinYear,
    PR.TrainingOpportunitiesTaken,
    SL1.SatisfactionLevel AS EnvironmentSatisfactionLevel,
    SL2.SatisfactionLevel AS JobSatisfactionLevel,
    SL3.SatisfactionLevel AS RelationshipSatisfactionLevel,
    SL4.SatisfactionLevel AS WorkLifeBalanceLevel,
    RL1.RatingLevel AS SelfRatingLevel,
    RL2.RatingLevel AS ManagerRatingLevel
FROM 
    PerformanceRating PR
LEFT JOIN 
    SatisfactionLevels SL1 ON PR.EnvironmentSatisfaction = SL1.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL2 ON PR.JobSatisfaction = SL2.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL3 ON PR.RelationshipSatisfaction = SL3.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL4 ON PR.WorkLifeBalance = SL4.SatisfactionID
LEFT JOIN 
    RatingLevels RL1 ON PR.SelfRating = RL1.RatingID
LEFT JOIN 
    RatingLevels RL2 ON PR.ManagerRating = RL2.RatingID;
"""

performancerating = read_sql_query(query)

performancerating.head()

Connection successful!


,PerformanceID,EmployeeID,ReviewDate,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,EnvironmentSatisfactionLevel,JobSatisfactionLevel,RelationshipSatisfactionLevel,WorkLifeBalanceLevel,SelfRatingLevel,ManagerRatingLevel
0,PR01,79F7-78EC,2013-01-02,1,0,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Exceeds Expectation,Exceeds Expectation
1,PR02,B61E-0F26,2013-01-03,1,3,Very Satisfied,Satisfied,Satisfied,Satisfied,Exceeds Expectation,Meets Expectation
2,PR03,F5E3-48BB,2013-01-03,3,2,Neutral,Satisfied,Very Satisfied,Neutral,Above and Beyond,Exceeds Expectation
3,PR04,0678-748A,2013-01-04,2,0,Very Satisfied,Neutral,Dissatisfied,Dissatisfied,Meets Expectation,Needs Improvement
4,PR05,541F-3E19,2013-01-04,1,0,Very Satisfied,Dissatisfied,Neutral,Satisfied,Exceeds Expectation,Meets Expectation


In [119]:
performancerating["ReviewDate"]=pd.to_datetime(performancerating["ReviewDate"], format='%Y-%m-%d', errors='coerce')

In [121]:
performancerating.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6709 entries, 0 to 6708
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   PerformanceID                    6709 non-null   object        
 1   EmployeeID                       6709 non-null   object        
 2   ReviewDate                       6709 non-null   datetime64[ns]
 3   TrainingOpportunitiesWithinYear  6709 non-null   int64         
 4   TrainingOpportunitiesTaken       6709 non-null   int64         
 5   EnvironmentSatisfactionLevel     6709 non-null   object        
 6   JobSatisfactionLevel             6709 non-null   object        
 7   RelationshipSatisfactionLevel    6709 non-null   object        
 8   WorkLifeBalanceLevel             6709 non-null   object        
 9   SelfRatingLevel                  6709 non-null   object        
 10  ManagerRatingLevel               6709 non-null   object     

# Summary statistics

In [122]:
numeric_performance = [
    "ReviewDate", 
    "TrainingOpportunitiesWithinYear",
    "TrainingOpportunitiesTaken",
]


categorical_performance = [
     "EnvironmentSatisfactionLevel", 
     "JobSatisfactionLevel", 
     "RelationshipSatisfactionLevel",
     "WorkLifeBalanceLevel", 
     "SelfRatingLevel",
     "ManagerRatingLevel"
]



In [123]:
# Summary statistics of numeric
performancerating[numeric_performance].describe()


,ReviewDate,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken
count,6709,6709.000000,6709.000000
mean,2019-04-14 05:12:04.936652544,2.012968,1.017290
min,2013-01-02 00:00:00,1.000000,0.000000
25%,2017-05-21 00:00:00,1.000000,0.000000
50%,2019-09-15 00:00:00,2.000000,1.000000
75%,2021-06-01 00:00:00,3.000000,2.000000
max,2022-12-31 00:00:00,3.000000,3.000000
std,NaN,0.820310,0.950316


In [132]:
# Summary statistics of categorical
performancerating[categorical_performance].describe().T

,count,unique,top,freq
EnvironmentSatisfactionLevel,6709,5,Neutral,2211
JobSatisfactionLevel,6709,5,Satisfied,1685
RelationshipSatisfactionLevel,6709,5,Satisfied,1690
WorkLifeBalanceLevel,6709,5,Satisfied,1706
SelfRatingLevel,6709,3,Meets Expectation,2290
ManagerRatingLevel,6709,4,Meets Expectation,2223


In [170]:
# Summary statistics of categorical
cat_summary_per = {}

for cat_c in categorical_performance:
    print(f"Summary statistics of: {cat_c}")
    print((performancerating[cat_c].value_counts(normalize=True)*100).to_string(index=True, header=True,float_format='%.2f%%'))
    print("\n")  


Summary statistics of: EnvironmentSatisfactionLevel
EnvironmentSatisfactionLevel
Neutral             32.96%
Satisfied           32.42%
Very Satisfied      30.50%
Dissatisfied         2.10%
Very Dissatisfied    2.03%


Summary statistics of: JobSatisfactionLevel
JobSatisfactionLevel
Satisfied           25.12%
Dissatisfied        24.95%
Neutral             24.61%
Very Satisfied      23.39%
Very Dissatisfied    1.94%


Summary statistics of: RelationshipSatisfactionLevel
RelationshipSatisfactionLevel
Satisfied           25.19%
Dissatisfied        25.15%
Neutral             24.24%
Very Satisfied      23.39%
Very Dissatisfied    2.04%


Summary statistics of: WorkLifeBalanceLevel
WorkLifeBalanceLevel
Satisfied           25.43%
Dissatisfied        25.37%
Neutral             24.89%
Very Satisfied      22.51%
Very Dissatisfied    1.80%


Summary statistics of: SelfRatingLevel
SelfRatingLevel
Meets Expectation      34.13%
Exceeds Expectation    33.33%
Above and Beyond       32.54%


Summary sta

____________________________


____________________________


____________________________

### Save dataset in one file Visual Dashboard.

In [94]:
query = """ 
SELECT 
    E.EmployeeID,
    E.FirstName,
    E.LastName,
    E.Gender,
    E.Age,
    E.BusinessTravel,
    E.Department,
    E.DistanceFromHome,
    E.State,
    E.Ethnicity,
    E.EducationField,
    E.JobRole,
    E.MaritalStatus,
    E.Salary,
    E.StockOptionLevel,
    E.OverTime,
    E.HireDate,
    E.Attrition,
    E.YearsAtCompany,
    E.YearsInMostRecentRole,
    E.YearsSinceLastPromotion,
    E.YearsWithCurrManager,
    EL.EducationLevel AS EducationLevel,
    PR.PerformanceID,
    PR.ReviewDate,
    PR.TrainingOpportunitiesWithinYear,
    PR.TrainingOpportunitiesTaken,
    SL1.SatisfactionLevel AS EnvironmentSatisfactionLevel,
    SL2.SatisfactionLevel AS JobSatisfactionLevel,
    SL3.SatisfactionLevel AS RelationshipSatisfactionLevel,
    SL4.SatisfactionLevel AS WorkLifeBalanceLevel,
    RL1.RatingLevel AS SelfRatingLevel,
    RL2.RatingLevel AS ManagerRatingLevel
FROM 
    Employees E
LEFT JOIN 
    EducationLevels EL ON E.Education = EL.EducationLevelID
LEFT JOIN 
    PerformanceRating PR ON E.EmployeeID = PR.EmployeeID
LEFT JOIN 
    SatisfactionLevels SL1 ON PR.EnvironmentSatisfaction = SL1.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL2 ON PR.JobSatisfaction = SL2.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL3 ON PR.RelationshipSatisfaction = SL3.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL4 ON PR.WorkLifeBalance = SL4.SatisfactionID
LEFT JOIN 
    RatingLevels RL1 ON PR.SelfRating = RL1.RatingID
LEFT JOIN 
    RatingLevels RL2 ON PR.ManagerRating = RL2.RatingID;
"""


data_employees = read_sql_query(query)


data_employees.head()

Connection successful!


,EmployeeID,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome,State,Ethnicity,EducationField,JobRole,MaritalStatus,Salary,StockOptionLevel,OverTime,HireDate,Attrition,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager,EducationLevel,PerformanceID,ReviewDate,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,EnvironmentSatisfactionLevel,JobSatisfactionLevel,RelationshipSatisfactionLevel,WorkLifeBalanceLevel,SelfRatingLevel,ManagerRatingLevel
0,001A-8F88,Christy,Jumel,Male,22,Some Travel,Technology,40,CA,White,Information Systems,Software Engineer,Married,27763.00,0,No,2021-09-05,No,1,0,1,0,Masters,None,None,NaN,NaN,None,None,None,None,None,None
1,005C-E0FB,Fin,O'Halleghane,Non-Binary,24,Frequent Traveller,Sales,17,CA,White,Marketing,Sales Executive,Married,56155.00,1,No,2017-08-26,No,5,2,2,0,Masters,PR4067,2020-06-17,1.0,2.0,Neutral,Neutral,Dissatisfied,Dissatisfied,Exceeds Expectation,Meets Expectation
2,005C-E0FB,Fin,O'Halleghane,Non-Binary,24,Frequent Traveller,Sales,17,CA,White,Marketing,Sales Executive,Married,56155.00,1,No,2017-08-26,No,5,2,2,0,Masters,PR5070,2021-06-17,1.0,1.0,Satisfied,Satisfied,Very Satisfied,Very Satisfied,Meets Expectation,Meets Expectation
3,005C-E0FB,Fin,O'Halleghane,Non-Binary,24,Frequent Traveller,Sales,17,CA,White,Marketing,Sales Executive,Married,56155.00,1,No,2017-08-26,No,5,2,2,0,Masters,PR6165,2022-06-17,3.0,0.0,Neutral,Satisfied,Very Satisfied,Satisfied,Exceeds Expectation,Exceeds Expectation
4,00A3-2445,Wyatt,Ziehm,Male,30,Some Travel,Technology,6,CA,Black or African American,Computer Science,Machine Learning Engineer,Married,126238.00,0,No,2012-03-08,No,10,3,6,6,High School,PR1165,2016-06-19,2.0,2.0,Satisfied,Very Satisfied,Satisfied,Very Satisfied,Exceeds Expectation,Meets Expectation


In [95]:
data_employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6899 entries, 0 to 6898
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   EmployeeID                       6899 non-null   object 
 1   FirstName                        6899 non-null   object 
 2   LastName                         6899 non-null   object 
 3   Gender                           6899 non-null   object 
 4   Age                              6899 non-null   int64  
 5   BusinessTravel                   6899 non-null   object 
 6   Department                       6899 non-null   object 
 7   DistanceFromHome                 6899 non-null   int64  
 8   State                            6899 non-null   object 
 9   Ethnicity                        6899 non-null   object 
 10  EducationField                   6899 non-null   object 
 11  JobRole                          6899 non-null   object 
 12  MaritalStatus       

In [97]:
data_employees.to_csv(f'{pth}06-All_Data_Employees.csv', index=False, encoding='utf-8-sig')


_____________________________________

## (2) Data Cleaning:
After building the data model, we proceeded with data cleaning and preprocessing. Here’s a summary of the key observations:

- The data had no missing values or unusual entries across all tables. Each field, such as age, salary, and years of experience, showed values within expected ranges.
- Data entries for categorical variables like gender, marital status, and job roles were consistent without any spelling or formatting issues.
- Date fields, such as the employee `HireDate` and `ReviewDate` in the performance review table, were in the correct format and adhered to the expected chronological order.
- Numeric fields, including `Salary`, `YearsAtCompany`, and `DistanceFromHome`, were confirmed to contain only valid numbers without any outliers or inconsistent values.

## Conclusion:
The dataset was thoroughly examined and found to be clean, consistent, and aligned with the designed data model. There were no missing values, illogical entries, or repeated values, ensuring data integrity across all tables. 


- **Python (pandas, Matplotlib)**: For detailed data Cleaning and visual inspection.

# END